In [1]:
import os
import torch
import numpy as np
from pathlib import Path
from torchvision import datasets
from torch.utils.data import DataLoader
import open_clip
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle

e:\Jain Irrigation\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
model,_, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='laion2b_s34b_b88k')
model = model.to(device)
model.eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [4]:
train_dir = Path(r"E:\Jain Irrigation\data\images\RMVBG_train")
fw_test_dir = Path(r"E:\Jain Irrigation\data\images\RMVBG_val")  # forward test

In [5]:
def extract_embeddings(folder_path, batch_size=16):
    dataset = datasets.ImageFolder(folder_path, transform=preprocess)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    all_embeddings, all_labels = [], []

    with torch.no_grad():
        for imgs, lbls in tqdm(loader, desc=f"Extracting from {folder_path.name}"):
            imgs = imgs.to(device)
            feats = model.encode_image(imgs)
            feats = feats / feats.norm(dim=-1, keepdim=True)
            all_embeddings.append(feats.cpu().numpy())
            all_labels.extend(lbls.numpy())

    embeddings = np.vstack(all_embeddings)
    labels = np.array(all_labels)
    class_names = dataset.classes
    return embeddings, labels, class_names


In [6]:
X_all, y_all, class_names = extract_embeddings(train_dir)
print("Total embeddings shape:", X_all.shape)

Extracting from RMVBG_train: 100%|██████████| 22/22 [00:11<00:00,  1.93it/s]

Total embeddings shape: (341, 512)


In [7]:
# --- Step 2: Split TRAIN into 70:30 train/test ---
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all,
    test_size=0.3,
    stratify=y_all,
    random_state=42
)
print("Train:", X_train.shape, "Test:", X_test.shape)

Train: (238, 512) Test: (103, 512)


In [8]:
X_fw, y_fw, _ = extract_embeddings(fw_test_dir)
print("Forward Test:", X_fw.shape)

Extracting from RMVBG_val: 100%|██████████| 6/6 [00:02<00:00,  2.07it/s]

Forward Test: (93, 512)


In [9]:
os.makedirs("embeddings", exist_ok=True)

with open("embeddings/rmvbg_train.pkl", "wb") as f:
    pickle.dump((X_train, y_train, class_names), f)

with open("embeddings/rmvbg_test.pkl", "wb") as f:
    pickle.dump((X_test, y_test, class_names), f)

with open("embeddings/rmvbg_forward.pkl", "wb") as f:
    pickle.dump((X_fw, y_fw, class_names), f)

print("✅ Saved embeddings to /embeddings folder")

✅ Saved embeddings to /embeddings folder


In [10]:
import pandas as pd
import numpy as np

# Example for forward test embeddings
df_fw = pd.DataFrame(X_fw)

# Add labels (optional)
df_fw["label"] = y_fw

# Save as CSV
df_fw.to_csv("embeddings/rmvbg_forward.csv", index=False)

print("✅ Saved embeddings/rmvbg_forward.csv")

✅ Saved embeddings/rmvbg_forward.csv
